In [1]:
# Import Neccessary Libraries:

import numpy as np
import pandas as pd
import torch
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
import pickle
from torch.optim.lr_scheduler import StepLR
import time
from PIL import Image
from torchsummary import summary
import time
import cv2

import seaborn as sns

pd.set_option('display.max_colwidth', -1)
pd.set_option("display.max_rows", 100000)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [2]:
# To mount drive:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Checking GPU:
print(torch.cuda.get_device_name(0))

Tesla T4


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
def get_data_from_drive(copy = True, extract = True):
  start = time.time()
  if copy:
    print("Copying Train Source from Drive.")
    !cp "/content/drive/MyDrive/DS5500 Data/nasa_tropical_storm_competition_train_source.tar" -r "/content/nasa_tropical_storm_competition_train_source.tar"
    print("Copying Test Source from Drive.")
    !cp "/content/drive/MyDrive/DS5500 Data/nasa_tropical_storm_competition_test_source.tar" -r "/content/nasa_tropical_storm_competition_test_source.tar"
    print("Copying Train Metadata from Drive.")
    !cp "/content/drive/MyDrive/DS5500 Data/train.csv" -r "/content/train.csv"
    print("Copying Test Metadata from Drive.")
    !cp "/content/drive/MyDrive/DS5500 Data/test.csv" -r "/content/test.csv"

  if extract:
    print("Extracting Training Images")
    !tar -xvf "/content/nasa_tropical_storm_competition_train_source.tar" -C "/content/"
    print("Extracting Test Images")
    !tar -xvf "/content/nasa_tropical_storm_competition_test_source.tar" -C "/content/"
  
  print("Data Ready! Time Taken: {:.4f}s".format(time.time() - start))

  
def get_image_paths(data, image_dir, folder_name):
    data["image_path"] = image_dir + "/" + folder_name + "_" + data["image_id"] + "/" + "image.jpg"
    data = data[data.columns[[0, 2, 5, 1, 3, 4]]]
    return data
    

In [11]:
def loadModel(model_path, model = None):
  state_dict = torch.load(model_path)
  model.load_state_dict(state_dict)
  return model

In [12]:
def get_data_in_intervals(metadata, interval):
  
  data_dict = dict()
  counter = 0
  for name, group in metadata.groupby(by="storm_id"):
    group = group.sort_values(by="relative_time").reset_index()
    # print(name, group.index)
    # removing storms with just one rgb image
    if group.shape[0] > 2*interval+1:
      for i in range(0, group.shape[0]):
        i2 = i-interval
        i3 = i2-interval
        if i3 >= 0 and i2 > 0:
          # print(i, i2, i3)
          lis = [group.image_path[i3], group.image_path[i2], group.image_path[i]]
          # storm_id = metadata.storm_id[i3]
          wind_speed = group.wind_speed[i]

          if counter not in data_dict.keys():
            data_dict[counter] = {"storm_id": name, "image_list": lis, "wind_speed": wind_speed}
          counter += 1

  df = pd.DataFrame.from_dict(data_dict, orient="index").reset_index().drop(columns="index")

  return df

In [7]:
class HurricaneImageDataset(Dataset):

    def __init__(self, metadata, transforms, add_color=False):
        self.metadata = metadata # new: with list of image paths, wind speed
        self.transforms = transforms
        self.add_color = add_color

    def __len__(self):
        return len(self.metadata)

    def convert_to_threeband(self, img_list):
      def add_rgb_to_images(img):
        # create 1 pixel red image
        red = np.zeros((1, 1, 3), np.uint8)
        red[:] = (0,0,255)
        # resize lut to 256 values
        # # create 1 pixel blue image
        blue = np.zeros((1, 1, 3), np.uint8)
        blue[:] = (255,0,0)
        # resize lut to 256 values
        # # create 1 pixel green image
        green = np.zeros((1, 1, 3), np.uint8)
        green[:] = (0,255,0)
        # append the three images
        lut = np.concatenate((red, green, blue), axis=0)
        lut = cv2.resize(lut, (1,256), interpolation=cv2.INTER_CUBIC)
        # apply lut
        result = cv2.LUT(img, lut)
        return result
      
      # Reading the three images
      img1 = cv2.imread(img_list[0], cv2.IMREAD_GRAYSCALE) # t - 2*interval
      img2 = cv2.imread(img_list[1], cv2.IMREAD_GRAYSCALE) # t - interval
      img3 = cv2.imread(img_list[2], cv2.IMREAD_GRAYSCALE) # t

      # Merge the three images to convert to three equal channels
      img = cv2.merge((img3, img2, img1))
      # img = img/255
      # print(img)

      # Add color if True
      if self.add_color:
        img = add_rgb_to_images(img)
        
      # return np.asarray(img)
      # return img
      return Image.fromarray(img)

    def __getitem__(self, index):
      if torch.is_tensor(index):
          index = index.tolist()

      # rgb_image = []
      image_paths = self.metadata["image_list"][index]
      hurricane_image = self.convert_to_threeband(image_paths)
      # for ip in image_paths:
      #   img = Image.open(image_path)
      #   rgb_image.append(img)
      # hurricane_image = io.imread(image_path)
      
      label = self.metadata["wind_speed"][index]

      if self.transforms:
          hurricane_image = self.transforms(hurricane_image)

      return hurricane_image, label

In [13]:
def get_image_features_from_pretrained_models(model, target_layer, feature_size, image):
  model.eval()
  target_layer = model.get_submodule(target_layer)
  embeddings = torch.zeros(feature_size)

  def copy_output_data(module, input, output):
    embeddings.copy_(output.flatten())

  hook = target_layer.register_forward_hook(copy_output_data)
  with torch.no_grad():
    model(image.unsqueeze(0))
  hook.remove()

  return embeddings

In [8]:
def extract_features_from_cnns(dataloader, og_df, model, target_layer, feature_size, device, filepath, save_df=True):
  df_list = list()
  # counter = 0
  print("Starting Feature Extraction...")
  for images, _ in tqdm(dataloader):
    # if counter == 5:
    #   break
    images = images.to(device)
    for enum, i in enumerate(images):
      img_features = get_image_features_from_pretrained_models(model, target_layer, feature_size, i)
      img_features = img_features.reshape(-1, feature_size)
      img_features = pd.DataFrame(img_features.numpy()).astype(float)
      # print(img_features.shape)
      df_list.append(img_features)
      # print(enum)
    #   if enum == 500:
    #     break
    # counter += 1
  print("Feature Extraction Completed.")
  print("Adding Storm ID, Wind Speed and Image List to Features Data...")
  features_df = pd.concat(df_list, ignore_index=True)
  features_df["storm_id"] = og_df["storm_id"]
  features_df["image_list"] = og_df["image_list"]
  features_df["wind_speed"] = og_df["wind_speed"]
  print("Size of Features Data: {}".format(features_df.shape))

  if save_df:
    print("Saving Features Data to Disk...")
    features_df.to_csv(filepath)
  print("Completed.")

  return features_df

In [ ]:
get_data_from_drive(copy=True, extract=True)

In [14]:
rs = np.random.RandomState(111) # creating a random state for reproducibility of random generators
data_dir = "/content/"
train_metadata_og = pd.read_csv("{}/train.csv".format(data_dir))
test_metadata_og = pd.read_csv("{}/test.csv".format(data_dir))
# msk = rs.rand(len(train_metadata)) < 0.8
# train_metadata2 = train_metadata[msk].reset_index().drop("index", axis=1)
# valid_metadata = train_metadata[~msk].reset_index().drop("index", axis=1)

train_folder_name = "nasa_tropical_storm_competition_train_source"
test_folder_name = "nasa_tropical_storm_competition_test_source"
train_image_dir = "{}/{}".format(data_dir, train_folder_name)
test_image_dir = "{}/{}".format(data_dir, test_folder_name)

train_metadata = get_image_paths(train_metadata_og, train_image_dir, train_folder_name)
test_metadata = get_image_paths(test_metadata_og, test_image_dir, test_folder_name)
# valid_metadata = get_image_paths(valid_metadata, train_image_dir, train_folder_name)

interval = 9
train_metadata = get_data_in_intervals(train_metadata, interval)
# valid_metadata = get_data_in_intervals(valid_metadata, interval)
test_metadata = get_data_in_intervals(test_metadata, interval)

from sklearn.model_selection import train_test_split

X,y = train_metadata[["storm_id", "image_list"]], train_metadata["wind_speed"]
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=101, stratify=train_metadata["storm_id"], shuffle=True)
# X_train, X_valid = pd.DataFrame(X_train), pd.DataFrame(X_valid)
# train_metadata = pd.concat([X_train, y_train], axis = 1).reset_index().drop("index", axis=1)
# valid_metadata = pd.concat([X_valid, y_valid], axis = 1).reset_index().drop("index", axis=1)


In [15]:
# X_train, y_train = train_metadata[["storm_id","image_list"]], train_metadata["wind_speed"]
# X_test, y_test = test_metadata[["storm_id","image_list"]], test_metadata["wind_speed"]

In [16]:
transform = transforms.Compose([
        # transforms.RandomResizedCrop(224),
        transforms.Resize((224, 224)),
        # transforms.RandomRotation(90),
        # transforms.Grayscale(num_output_channels=1),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        # transforms.CenterCrop(90),
        transforms.ToTensor(),
        # transforms.Normalize([0.5], [0.5])
        # transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [17]:
trainset = HurricaneImageDataset(train_metadata, transform)
testset = HurricaneImageDataset(test_metadata, transform)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=True)

In [19]:
densenet161 = models.densenet161(pretrained = True)
model = densenet161.to(device)
# feature_size = model.classifier.in_features
feature_size = 2352
target_layer = "features.denseblock4.denselayer24.conv2"
filepath = "/content/drive/MyDrive/DS5500 Data/df_train_densenet161_hurricane_features.csv"
image_features_df_train = extract_features_from_cnns(trainloader, train_metadata, model, target_layer, feature_size, device, filepath)

Starting Feature Extraction...


100%|██████████| 482/482 [34:55<00:00,  4.35s/it]


Feature Extraction Completed.
Adding Disease Vector and Image Index to Features Data...
Size of Features Data: (61637, 2355)
Saving Features Data to Disk...
Completed.


In [21]:
image_features_df_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2315,2316,2317,2318,2319,2320,2321,2322,2323,2324,2325,2326,2327,2328,2329,2330,2331,2332,2333,2334,2335,2336,2337,2338,2339,2340,2341,2342,2343,2344,2345,2346,2347,2348,2349,2350,2351,storm_id,image_list,wind_speed
0,-0.003550,-0.007286,-0.009102,-0.009400,-0.002762,-0.002491,0.001555,-0.007246,-0.015684,-0.013834,-0.011189,0.002913,-0.000631,0.003968,-0.007320,-0.017400,-0.012896,-0.009794,0.007933,0.001283,0.004546,-0.009308,-0.022382,-0.015025,-0.013189,0.005586,0.000678,0.002218,-0.004747,-0.013956,-0.016703,-0.014897,-0.001563,0.001457,0.003593,-0.005106,-0.011663,-0.011059,-0.007618,0.002660,...,-0.000815,0.000195,-0.000520,-0.010003,-0.010994,-0.015710,-0.010773,-0.004214,-0.001359,-0.001921,-0.008093,-0.011520,-0.014650,-0.009952,-0.004935,0.001350,-0.002437,-0.004116,-0.004652,-0.006399,-0.006224,-0.005915,0.001235,-0.004190,-0.006944,-0.007803,-0.005855,-0.006044,-0.004887,-0.000203,-0.002486,-0.003820,-0.004796,-0.004386,-0.004609,-0.004294,-0.001626,abs,"[/content//nasa_tropical_storm_competition_train_source/nasa_tropical_storm_competition_train_source_abs_000/image.jpg, /content//nasa_tropical_storm_competition_train_source/nasa_tropical_storm_competition_train_source_abs_009/image.jpg, /content//nasa_tropical_storm_competition_train_source/nasa_tropical_storm_competition_train_source_abs_018/image.jpg]",64
1,0.001127,0.004741,0.010198,0.007915,0.000789,-0.003223,-0.002085,0.001353,0.006295,0.010862,0.010055,0.003702,-0.000270,-0.001342,0.004990,0.006383,0.008112,0.007953,0.006756,0.010713,0.004686,0.001326,-0.002544,-0.004608,-0.002364,0.001053,0.009612,0.005772,0.000059,-0.008911,-0.015743,-0.012407,-0.004361,0.006059,0.002842,0.000356,-0.006457,-0.022325,-0.021231,-0.014759,...,-0.007073,-0.001472,-0.000112,-0.007446,-0.011731,-0.016133,-0.015982,-0.015153,-0.008647,-0.000959,-0.001419,-0.006140,-0.010847,-0.015536,-0.010103,-0.008070,-0.001413,0.002018,0.000225,-0.000540,-0.006573,-0.003265,-0.005628,-0.000667,0.004782,0.005082,0.008407,0.003016,0.001885,-0.001224,-0.000042,0.003036,0.005135,0.006542,0.004115,0.000852,0.000636,abs,"[/content//nasa_tropical_storm_competition_train_source/nasa_tropical_storm_competition_train_source_abs_001/image.jpg, /content//nasa_tropical_storm_competition_train_source/nasa_tropical_storm_competition_train_source_abs_010/image.jpg, /content//nasa_tropical_storm_competition_train_source/nasa_tropical_storm_competition_train_source_abs_019/image.jpg]",64
2,0.011335,0.009224,0.007510,-0.005410,-0.006022,-0.008802,-0.002640,0.007572,0.005992,0.009079,0.001824,0.003871,-0.003377,-0.000032,-0.005963,-0.008578,0.000786,0.001050,0.007313,0.012519,0.013238,-0.006640,-0.007523,-0.006421,-0.003679,-0.000196,0.009032,0.006604,0.006865,0.005178,-0.000733,-0.010232,-0.013174,-0.001420,0.000889,0.002165,-0.000593,-0.007320,-0.015600,-0.020654,...,-0.011209,-0.006633,-0.006711,-0.013310,0.002829,0.003188,0.002511,-0.013178,-0.012625,-0.004038,-0.014142,-0.002199,-0.008778,-0.001604,-0.014719,-0.011130,0.001344,-0.004044,0.009858,0.002543,0.004326,-0.010100,-0.009286,0.002643,-0.001990,-0.003479,-0.011067,-0.010068,-0.008374,-0.004312,0.001811,0.001397,0.000819,-0.002579,-0.005391,-0.004161,-0.003541,abs,"[/content//nasa_tropical_storm_competition_train_source/nasa_tropical_storm_competition_train_source_abs_002/image.jpg, /content//nasa_tropical_storm_competition_train_source/nasa_tropical_storm_competition_train_source_abs_011/image.jpg, /content//nasa_tropical_storm_competition_train_source/nasa_tropical_storm_competition_train_source_abs_020/image.jpg]",64
3,0.000995,0.001349,-0.001129,-0.002661,-0.003483,-0.004381,-0.003357,0.004359,0.000799,-0.005693,-0.014941,-0.014152,-0.013570,-0.004672,-0.000433,-0.008061,-0.013867,-0.028288,-0.025916,-0.016445,-0.004700,-0.000334,-0.010856,-0.012212,-0.029193,-0.023812,-0.020287,-0.007122,-0.004004,-0.016259,-0.017722,-0.030165,-0.01

In [20]:
# densenet161 = models.densenet161(pretrained = True)
# model = densenet161.to(device)
# # feature_size = model.classifier.in_features
feature_size = 2352
target_layer = "features.denseblock4.denselayer24.conv2"
filepath = "/content/drive/MyDrive/DS5500 Data/df_test_densenet161_hurricane_features.csv"
image_features_df_test = extract_features_from_cnns(testloader, test_metadata, model, target_layer, feature_size, device, filepath)

Starting Feature Extraction...


100%|██████████| 297/297 [21:27<00:00,  4.33s/it]


Feature Extraction Completed.
Adding Disease Vector and Image Index to Features Data...
Size of Features Data: (37913, 2355)
Saving Features Data to Disk...
Completed.


In [22]:
image_features_df_test.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2315,2316,2317,2318,2319,2320,2321,2322,2323,2324,2325,2326,2327,2328,2329,2330,2331,2332,2333,2334,2335,2336,2337,2338,2339,2340,2341,2342,2343,2344,2345,2346,2347,2348,2349,2350,2351,storm_id,image_list,wind_speed
0,-0.001160,-0.001120,-0.000784,-0.001614,-0.000984,-0.005864,-0.002829,-0.002140,-0.005242,-0.007274,-0.006234,-0.003373,-0.007780,-0.003628,-0.001577,-0.003458,-0.004006,-0.004155,-0.003297,-0.000960,-0.000876,-0.001937,-0.005701,-0.008731,-0.008187,-0.007387,-0.002073,-0.000947,-0.001438,-0.005093,-0.007195,-0.009251,-0.006791,-0.002603,0.000078,-0.000967,-0.005036,-0.004278,-0.006324,-0.002808,...,-0.000326,0.001354,-0.002818,-0.010552,-0.011218,-0.007816,-0.000915,-0.000991,-0.000258,-0.003556,-0.012778,-0.010968,-0.006179,0.001809,0.002313,-0.000197,-0.000791,-0.008398,-0.001464,-0.001652,0.004651,0.003335,0.001004,0.000476,-0.001207,0.000548,-0.000482,0.000406,0.002011,0.002124,0.001568,0.002081,0.003531,0.001224,0.000862,0.001087,0.001970,acd,"[/content//nasa_tropical_storm_competition_test_source/nasa_tropical_storm_competition_test_source_acd_123/image.jpg, /content//nasa_tropical_storm_competition_test_source/nasa_tropical_storm_competition_test_source_acd_132/image.jpg, /content//nasa_tropical_storm_competition_test_source/nasa_tropical_storm_competition_test_source_acd_141/image.jpg]",30
1,-0.000089,-0.001554,0.004143,0.002703,0.005077,0.000657,0.000336,-0.002449,-0.001684,0.000346,-0.002526,-0.002878,-0.004747,-0.000328,-0.003425,-0.000875,-0.005664,-0.004200,-0.007738,-0.003876,-0.001295,-0.010969,-0.017663,-0.029446,-0.019596,-0.013726,-0.002090,0.000210,-0.009588,-0.017769,-0.031659,-0.022561,-0.012469,-0.002871,0.000684,-0.007624,-0.018687,-0.027980,-0.026662,-0.014978,...,-0.001726,0.000289,-0.001987,0.001181,0.006113,0.008666,0.005789,-0.000995,-0.002023,-0.004154,-0.002494,-0.002805,0.001704,-0.001232,-0.001076,-0.002790,-0.004068,-0.002446,-0.003403,0.001329,0.000184,0.003119,-0.000477,-0.001006,-0.003252,-0.009476,-0.010515,-0.011167,-0.001740,-0.001395,0.000222,-0.002708,-0.006217,-0.005604,-0.004774,-0.000270,0.000290,acd,"[/content//nasa_tropical_storm_competition_test_source/nasa_tropical_storm_competition_test_source_acd_124/image.jpg, /content//nasa_tropical_storm_competition_test_source/nasa_tropical_storm_competition_test_source_acd_133/image.jpg, /content//nasa_tropical_storm_competition_test_source/nasa_tropical_storm_competition_test_source_acd_142/image.jpg]",30
2,-0.005360,-0.009921,-0.010257,-0.008396,0.002953,0.004118,0.006383,-0.009008,-0.011066,-0.012372,-0.007451,0.001754,0.002156,0.004987,-0.004935,0.019998,0.040106,0.047020,0.030865,0.004715,0.005499,-0.006499,0.015137,0.021516,0.029219,0.012524,-0.005021,-0.002439,-0.000197,0.020488,0.033468,0.034150,0.014331,-0.006159,-0.003485,0.005007,0.005293,-0.002657,-0.008902,-0.013793,...,-0.004718,-0.003137,-0.006321,-0.009836,-0.018162,-0.006242,-0.005526,0.002682,-0.001629,-0.007637,-0.011976,-0.019682,-0.013614,-0.010106,-0.001326,-0.000488,-0.010213,-0.014543,-0.016834,-0.007656,-0.002997,0.000385,0.001459,-0.010772,-0.018829,-0.026030,-0.022461,-0.014275,-0.006811,-0.000131,-0.005381,-0.011197,-0.013254,-0.011409,-0.004929,-0.002992,-0.000493,acd,"[/content//nasa_tropical_storm_competition_test_source/nasa_tropical_storm_competition_test_source_acd_125/image.jpg, /content//nasa_tropical_storm_competition_test_source/nasa_tropical_storm_competition_test_source_acd_134/image.jpg, /content//nasa_tropical_storm_competition_test_source/nasa_tropical_storm_competition_test_source_acd_143/image.jpg]",30
3,-0.005878,-0.006456,-0.001434,-0.002630,0.000808,-0.002334,0.000972,-0.009640,-0.014201,-0.009425,-0.011651,-0.008515,-0.009556,-0.000620,-0.005202,0.002025,0.011314,0.001002,-0.003035,-0.013420,0.001255,-0.002305,-0.001370,-0.004853,-0.013432,-0.012478,-0.018716,-0.005088,0.000197,0.003062,0.006392,0.000925,-0.003322